# DataSet Preparation Utils
by Wael Radwan

#### Library used to prepare datasets from data
continans various tools to prepare the training dataset and testing dataset

mainly used to randomize files of the dataset
divide the dataset into traininig and testing dataset
randomize the test files

<b> the main idea here is to load the dataset into classified directories then use this utility to create train/test dataset with the randomization </b>

In [1]:
import os
import re
import random
import ntpath
import shutil
import pandas as pd

#### Input description
input to this tool is a dataset of files divided into classes in the following order
- class_1:
    - file_1
    - file_2
    - .
    - .
- class_2
    - file_1
    - file_2
    - .
    - .
- .
- .


- class_n 
    - file_1
    - file_2
    - .
    - .


#### Output description
output shoud be the data divided into train and test sets  in the following order

- train
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
- test
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
    - class_label.csv (a csv file contains the filename and the label so it can be used for testing comparison)

#### create panda from the files dataset
mainly used for the test dataset to generate the file name and the label based on the label criteria
randomize is set to true if the files are ranomized in the dataset instead of ordered

In [108]:
def list_files_with_labels(path, randomize=False):
    files = sorted(os.listdir(path))
    file = [os.path.splitext(x)[0] for x in files]
    ext = [os.path.splitext(x)[-1] for x in files]
    class_name = ntpath.basename(path)
    

    # create panda dataframe for file sorted by their name

    dataset = pd.DataFrame(data={"file":file,"ext":ext, "label":class_name})
    dataset = dataset[['file', 'ext', 'label']]

    # if randomize is set to True then shuffle the dataset dataframe
    if (randomize) :
        dataset= dataset.sample(frac=1).reset_index()
        #dataset = dataset.sample(frac=1).reset_index(drop=True)

    return dataset


#### create list of the dataset with new name for each file based on a sequence
randomize or order the files in the dataset then create a new name based on sequence number

- path      : path to the class
- seqlen    : is the lenght of the sequence so if seqlen = 3 the new file names follow the sequence 001, 002, 003, ...
- randomize : either to randomize the files in the class set or not (default is not to randomize) 
- retrun panda structure with new file name 

In [109]:
def list_with_new_file_name_seq(path,  seqlen, randomize=False,):
    df = []  
    dataset = list_files_with_labels( path , randomize)
    
    # recreate index in case the dataset has been randomize
    # so the seq_number will be reflected in out file_name
    df = dataset.sample(frac=1).reset_index(drop=True)
    
    # new file name will be classid_seq(of seq_len).ext
    # for example 
    # 1_0001.mp3
    # 1_0002.mp3
    df['newfilename'] = df['label']+'_'+(df.index + 1).astype(str).str.zfill(seqlen) + df.ext
    
    # df.index = df.index.map(str) 
    return df

In [114]:
path='/ds/dataset/quran/999'
dataset = list_files_with_labels( path,randomize=True)
data = list_with_new_file_name_seq( path,3)

In [116]:
data

,file,ext,label,newfilename
0,112_189,.mp3,999,999_001.mp3
1,112_218,.mp3,999,999_002.mp3
2,112-Al-ikhlas_58,.mp3,999,999_003.mp3
3,112_87,.mp3,999,999_004.mp3
4,112_188,.mp3,999,999_005.mp3
5,112-Al-ikhlas_84,.mp3,999,999_006.mp3
6,112_168,.mp3,999,999_007.mp3
7,112_88,.mp3,999,999_008.mp3
8,112-Al-ikhlas_48,.mp3,999,999_009.mp3
9,112_98,.mp3,999,999_010.mp3


#### Rename files 
after creating the dataframe with proposed renamed files then do the rename files in the dataset
then rename all the files to _tmp files in order not to overwrite files that already exist 


In [33]:
def rename_files_dataset(path, seqlen):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_with_new_file_name_seq( path,seqlen)

    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        tmp_file_name = path+'/'+row['newfilename'] + '_tmp'
        orig_file_name =  path+'/'+row['newfilename'] 
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        tmp_file_names.append(tmp_file_name)
        
    # now remove _tmp f    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
    return

In [34]:
rename_files_dataset(path , 4)

rename /ds/dataset/quran/999/999_0001.mp3 to /ds/dataset/quran/999/999_0001.mp3_tmp
rename /ds/dataset/quran/999/999_0002.mp3 to /ds/dataset/quran/999/999_0002.mp3_tmp
rename /ds/dataset/quran/999/999_0003.mp3 to /ds/dataset/quran/999/999_0003.mp3_tmp
rename /ds/dataset/quran/999/999_0004.mp3 to /ds/dataset/quran/999/999_0004.mp3_tmp
rename /ds/dataset/quran/999/999_0005.mp3 to /ds/dataset/quran/999/999_0005.mp3_tmp
rename /ds/dataset/quran/999/999_0006.mp3 to /ds/dataset/quran/999/999_0006.mp3_tmp
rename /ds/dataset/quran/999/999_0007.mp3 to /ds/dataset/quran/999/999_0007.mp3_tmp
rename /ds/dataset/quran/999/999_0008.mp3 to /ds/dataset/quran/999/999_0008.mp3_tmp
rename /ds/dataset/quran/999/999_0009.mp3 to /ds/dataset/quran/999/999_0009.mp3_tmp
rename /ds/dataset/quran/999/999_0010.mp3 to /ds/dataset/quran/999/999_0010.mp3_tmp
rename /ds/dataset/quran/999/999_0011.mp3 to /ds/dataset/quran/999/999_0011.mp3_tmp
rename /ds/dataset/quran/999/999_0012.mp3 to /ds/dataset/quran/999/999_0012.

##### splitt the dataset into training and test
here split the dataset into training and testing 
- path : path to the class
- train_path : path to the train dir
- test_path : path to the test dir
- train_files_count : is the number of files in each class to be moved to the training dir
- randomize : either to randomize the files in the class set or not (default is not to randomize)

the output will be copying the data into train and test directories

In [122]:
def split_into_train_test( path, train_path, test_path,train_files_count, randomize=True):
    # read the files in the origianl path ordered by there name
    data = list_files_with_labels( path,randomize)
    # copy the dir name of the class 
    # this only copy the current directory name (without the path)
    dir_name = ntpath.basename(path)
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        orig_file_name =  path+'/'+row['file'] + row['ext']
        if (index<train_files_count):
            tmp_file_name = train_path+'/'+row['file']  + row['ext']
        else :
            tmp_file_name = test_path+'/'+row['file']  + row['ext']
       
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
    return

In [123]:
train_path='/ds/dataset/quran/train'
test_path='/ds/dataset/quran/test'
split_into_train_test( path, train_path, test_path,10)

rename /ds/dataset/quran/999/112_58.mp3 to /ds/dataset/quran/train/112_58.mp3
rename /ds/dataset/quran/999/112_81.mp3 to /ds/dataset/quran/train/112_81.mp3
rename /ds/dataset/quran/999/112-Al-ikhlas_58.mp3 to /ds/dataset/quran/train/112-Al-ikhlas_58.mp3
rename /ds/dataset/quran/999/112_181.mp3 to /ds/dataset/quran/train/112_181.mp3
rename /ds/dataset/quran/999/112-Al-ikhlas_38.mp3 to /ds/dataset/quran/train/112-Al-ikhlas_38.mp3
rename /ds/dataset/quran/999/112_uP_bY_mUSLEm.Ettounssi_28.mp3 to /ds/dataset/quran/train/112_uP_bY_mUSLEm.Ettounssi_28.mp3
rename /ds/dataset/quran/999/112_uP_bY_mUSLEm_38.mp3 to /ds/dataset/quran/train/112_uP_bY_mUSLEm_38.mp3
rename /ds/dataset/quran/999/112_84.mp3 to /ds/dataset/quran/train/112_84.mp3
rename /ds/dataset/quran/999/112-Al-ikhlas_48.mp3 to /ds/dataset/quran/train/112-Al-ikhlas_48.mp3
rename /ds/dataset/quran/999/112_80.mp3 to /ds/dataset/quran/train/112_80.mp3
rename /ds/dataset/quran/999/112_148.mp3 to /ds/dataset/quran/test/112_148.mp3
rename 

###### to be continued
from here we will continue by adding more features
- reshuffle the test and generate a csv description file for test to be used kaggle 